In [ ]:
import parse

parse.read_file("data/subdivision4.txt")


In [ ]:
import pylab as pl
import matplotlib.pyplot as plt
import matplotlib.collections as mc
from parse import root, segment, halfedge, rand_segment, vert_segment, bound_box, vertex_x, vertex_y, vertex, point, point_x, point_y

fig, ax = pl.subplots()

segment_vertical_plot = []
'''vertical lines'''
for s in vert_segment:
	stt = (s.vsx, s.vsy)
	end = (s.vex, s.vey)
	segment_vertical_plot.append([stt, end])
ax.add_collection(mc.LineCollection(segment_vertical_plot, linestyles='dashed', colors=('black')))

'''line segments'''
segment_plot = []
for s in halfedge:
	stt = (s.vsx, s.vsy)
	end = (s.vex, s.vey)
	segment_plot.append([stt, end])
ax.add_collection(mc.LineCollection(segment_plot, colors=('blue')))

'''bounding box'''
bound_box_plot = []
for s in bound_box:
	stt = (s.vsx, s.vsy)
	end = (s.vex, s.vey)
	bound_box_plot.append([stt, end])
ax.add_collection(mc.LineCollection(bound_box_plot, colors=('black')))

ax.set_xlim(0, 10)
ax.autoscale()
ax.margins(0.1)

for v in vertex:
	plt.text(v.x-0.015, v.y+0.25, "v"+str(v.idx))
plt.scatter(vertex_x, vertex_y, color="red", s=50)

for p in point:
	plt.text(p.x-0.15, p.y-0.5, "p"+str(p.idx))
plt.scatter(point_x, point_y, color="green", s=20)

# plt.savefig('trapzodialMap.png', dpi=1000)
plt.show()

In [ ]:
from DoublyConnectedEdgeList import Vertex, Face, HalfEdge, Trapezoid
from TrapezoidalMap import TrapezoidMap, display, Node
from DirectedAcyclicGraph import Node
from parse import root, segment, vertex, bound_box

tpm = TrapezoidMap(root)

seg = segment[:1]

vtx = tpm.build_trapezoid_map(seg)
pt = Vertex(2, 3, idx=10)
final = TrapezoidMap.find_point(tpm.root, pt)
print(final, final.path, final.data.top.face, final.data.bot.face)
# print(final.parent, final.parent.path)

display.show_map(vtx, seg, bound_box)


In [ ]:
from TrapezoidalMap import TrapezoidMap
from DoublyConnectedEdgeList import Vertex, Face, HalfEdge, Trapezoid
import networkx as nx

box_t = bound_box[0]
box_b = bound_box[1]
box_l = bound_box[2]
box_r = bound_box[3]
box_trap = Trapezoid(box_t, box_b, box_t.pl, box_b.pr)

map = nx.DiGraph()
map.add_node(box_trap)

n = [n for n,d in map.in_degree() if d==0]
# print(isinstance(n[0], Trapezoid))

import matplotlib.pyplot as plt
pos = nx.nx_agraph.graphviz_layout(map, prog="dot")
nx.draw_networkx(map, pos=pos, with_labels=True)
plt.show()



In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

map = nx.DiGraph()

for n in rand_segment:
	map.add_node(n)

from DoublyConnectedEdgeList import Vertex, Face, HalfEdge, Trapezoid
box_t = bound_box[0]
box_b = bound_box[1]
box_l = bound_box[2]
box_r = bound_box[3]
box_trap = Trapezoid(box_t, box_b, box_t.pl, box_b.pr)
map.add_node(box_trap)

for i in range(len(rand_segment)-1):
	map.add_edge(rand_segment[i], rand_segment[i+1])


leaf = [x for x in map.nodes() if map.out_degree(x)==0]
print(leaf[0])
pred = map.predecessors(leaf[0])
print(list(pred)[0])

# g.add_edge(1, 2)
# g.add_edge(2, 3)
# g.add_edge(3, 4)
# g.add_edge(1, 4)
# g.add_edge(1, 5)
# g.add_edge(5, 6)
# g.add_edge(5, 7)
# g.add_edge(4, 8)
# g.add_edge(3, 8)

# g.add_edge(5, 6)
# g.remove_node(6)
# g.add_node(6)
pos = nx.nx_agraph.graphviz_layout(map, prog="dot")
nx.draw_networkx(map, pos=pos, with_labels=True)
plt.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def create_DAG(dag):
	graph = nx.DiGraph()
	for v in dag.keys():
		graph.add_node(v)
	for n in dag.items():
		for w in n[1]:
			# print("n[0]: ", n[0], " | n[1]:", n[1], "w: ", w)
			graph.add_edge(n[0], w)
	return graph

dag = {
	's1': ['v', 't1','w'],
	's2': ['t1','s1'],
	's3': ['v','w'],
	's4': ['x','y'],
	'x': ['v','w'],
	'v': ['t1', 'w'],
	'w': ['y','t1','t2'],
	'y': ['v','t1','t2'],
	't1': [],
	't2': []
	}

map = create_DAG(dag)
pos = nx.nx_agraph.graphviz_layout(map, prog="dot")
nx.draw_networkx(map, pos=pos, with_labels=True)
plt.show()

# leaf = [x for x in map.nodes() if map.out_degree(x)==0 and map.in_degree(x)==2]
leaf = [x for x in map.nodes() if map.out_degree(x)==0]

print(leaf)

